### **Linear Chain**

In [ ]:
import os
from langchain.chat_models import init_chat_model
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate


os.getenv("GOOGLE_API_KEY")

In [13]:
llm = init_chat_model("google_genai:gemini-2.5-flash", temperature=0)

In [14]:
promt = ChatPromptTemplate.from_messages(
    [
    ("system" , "You are a rude ai"),
    ("user","give angry reply about {input}")
])

In [15]:
parser = StrOutputParser()

In [ ]:
chain = promt | llm 
topic = input()
for chunk in chain.stream({"input":topic}):
    print(chunk.content, flush=True)

### **Instagram post chain**

In [ ]:
chat_promt = ChatPromptTemplate.from_messages([
    ("system","You are a perfect post maker creativily"),
    ("user","Create me a post for the topic {input}")
])

In [ ]:
from langchain_core.runnables import RunnableLambda

def convert(data : str) -> dict:
    return {"text":data}

convertToDict = RunnableLambda(convert)

In [ ]:
from langchain_core.runnables import RunnableLambda

def dict_to_str(data : str) ->str:
    return data["text"]

convertToStr = RunnableLambda(dict_to_str)

In [ ]:
chain = chat_promt | llm | parser | convertToDict  |convertToStr| llm | parser
chain.invoke({"input" : input()})

'This is an **absolutely beautiful and incredibly well-crafted post!** It hits all the marks you aimed for: creative, engaging, and deeply heartfelt.\n\nHere\'s why it works so well:\n\n*   **Visual Prompt:** Starting with the suggestion for a warm, evocative image immediately sets the right tone and helps readers visualize the feeling of love before they even read a word.\n*   **Catchy & Evocative Title:** "✨ Love: The Unseen Thread That Connects Us All ✨" is brilliant. The metaphor is gentle yet powerful, and the emojis add a touch of warmth and visual appeal.\n*   **Heartfelt & Relatable Language:** You\'ve done an excellent job of defining love not just as a feeling, but through its manifestations. Phrases like "quiet understanding in a shared glance," "fierce loyalty," "comforting purr," and "gentle hand that helps you up" paint vivid, relatable pictures that resonate deeply.\n*   **Emphasis on Small Acts:** Highlighting that "Love isn\'t always grand gestures; often, it\'s in the

### **PARALLEL CHAIN**

In [2]:
promt_template = ChatPromptTemplate.from_messages([
    ("system", "You are a movie describer"),
    ("user","Tell me short explain about the movie {movie}")
])

promt_template

ChatPromptTemplate(input_variables=['movie'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a movie describer'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['movie'], input_types={}, partial_variables={}, template='Tell me short explain about the movie {movie}'), additional_kwargs={})])

In [3]:
from langchain_core.runnables import RunnableLambda

def convertToDict(text : str) -> dict:
    return {"text" : text}

convertRunnable = RunnableLambda(convertToDict)

In [4]:
insta_template = ChatPromptTemplate.from_messages([
    ("system", "You are a movie describer for instagram post"),
    ("user","Generate me post in instagram for the topic {text}")
]) 

llm = init_chat_model("google_genai:gemini-2.5-flash", temperature=0.5) 

str_parser = StrOutputParser()

insta_chain = insta_template | llm | str_parser

In [5]:
linkedin_template = ChatPromptTemplate.from_messages([
    ("system", "You are a movie describer for linkedin post"),
    ("user","Generate me post in linkedin for the topic {text}")
]) 

llm = init_chat_model("google_genai:gemini-2.5-flash", temperature=0.5) 

str_parser = StrOutputParser()

linkedin_chain = linkedin_template | llm | str_parser

In [6]:
from langchain_core.runnables import RunnableParallel

final_chain = promt_template | llm | str_parser | convertRunnable | RunnableParallel({"linkedin" : linkedin_chain, "Instagram" : insta_chain})

In [7]:
final_chain.invoke({"movie" : "RRR"})

{'linkedin': 'Here are a few options for a LinkedIn post about RRR, playing with slightly different angles:\n\n---\n\n**Option 1: Focusing on Vision & Impact**\n\nIf you\'re looking for a cinematic experience that transcends boundaries, look no further than S.S. Rajamouli\'s epic **RRR**.\n\nMore than just an action-drama, this film is a masterclass in visionary storytelling and grand-scale execution. Set in 1920s colonial India, it brilliantly fictionalizes the lives of legendary revolutionaries **Alluri Sitarama Raju (Ram Charan)** and **Komaram Bheem (Jr NTR)**.\n\nWhat makes RRR truly resonate?\n💥 **Spectacular Action:** Redefining "over-the-top" with sequences that will leave you breathless.\n❤️ **Powerful Emotion:** A deep, human core that grounds the larger-than-life narrative, particularly through the unbreakable bond of friendship formed between its protagonists.\n✨ **Unforgettable Vision:** Rajamouli\'s ability to craft a world where historical inspiration meets breathtaking 